In [1]:
import cntk.tests.test_utils
from sklearn import cross_validation
import cntk as C
from cntk.layers import Convolution, MaxPooling, Dense, BatchNormalization
from cntk.initializer import glorot_uniform
from cntk.ops import relu, sigmoid, input_variable, softmax
from cntk.learners import learning_rate_schedule, UnitType
from cntk.device import try_set_default_device, gpu, all_devices
print(all_devices())
try_set_default_device(gpu(0))
cntk.tests.test_utils.set_device_from_pytest_env() # (only needed for our build system)
C.cntk_py.set_fixed_random_seed(1) # fix the random seed so that LR examples are repeatable

from datetime import datetime
import numpy as np
import sys
import os
import math
from datetime import datetime
from pytz import timezone
from matplotlib import pyplot as plt
%matplotlib inline
from collections import defaultdict
import itertools

(GPU[0] Tesla M60, CPU)


/home/anaconda3/envs/cntk36/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [25]:
def UpSampling2D(x):
    xr = C.reshape(x, (x.shape[0], x.shape[1], 1, x.shape[2], 1))
    xx = C.splice(xr, xr, axis=-1) # axis=-1 refers to the last axis
    xy = C.splice(xx, xx, axis=-3) # axis=-3 refers to the middle axis
    r = C.reshape(xy, (x.shape[0], x.shape[1] * 2, x.shape[2] * 2))

    return r

def create_model(input, num_classes):
    conv1 = Convolution((3,3), 32, init=glorot_uniform(), activation=relu, pad=True)(input)
#     conv1 = BatchNormalization(map_rank=1)(conv1)
    conv1 = Convolution((3,3), 32, init=glorot_uniform(), activation=relu, pad=True)(conv1)
#     conv1 = BatchNormalization(map_rank=1)(conv1)
    pool1 = MaxPooling((2,2), strides=(2,2))(conv1)

    conv2 = Convolution((3,3), 64, init=glorot_uniform(), activation=relu, pad=True)(pool1)
    conv2 = Convolution((3,3), 64, init=glorot_uniform(), activation=relu, pad=True)(conv2)
#     conv2 = BatchNormalization(map_rank=1)(conv2)
    pool2 = MaxPooling((2,2), strides=(2,2))(conv2)
    
    conv3 = Convolution((3,3), 128, init=glorot_uniform(), activation=relu, pad=True)(pool2)
#     conv3 = BatchNormalization(map_rank=1)(conv3)
    conv3 = Convolution((3,3), 128, init=glorot_uniform(), activation=relu, pad=True)(conv3)
#     conv3 = BatchNormalization(map_rank=1)(conv3)
    pool3 = MaxPooling((2,2), strides=(2,2))(conv3)
    
    conv4 = Convolution((3,3), 256, init=glorot_uniform(), activation=relu, pad=True)(pool3)
#     conv4 = BatchNormalization(map_rank=1)(conv4)
    conv4 = Convolution((3,3), 256, init=glorot_uniform(), activation=relu, pad=True)(conv4)
    conv4 = BatchNormalization(map_rank=1)(conv4)
#     pool4 = MaxPooling((2,2), strides=(2,2))(conv4)
    
    conv5 = Convolution((3,3), 512, init=glorot_uniform(), activation=relu, pad=True)(conv4)
#     conv5 = BatchNormalization(map_rank=1)(conv5)
    conv5 = Convolution((3,3), 512, init=glorot_uniform(), activation=relu, pad=True)(conv5)
#     conv5 = BatchNormalization(map_rank=1)(conv5)
    
 #   up6 = C.splice(UpSampling2D(conv5), conv4, axis=0)
    conv6 = Convolution((3,3), 256, init=glorot_uniform(), activation=relu, pad=True)(conv5)
#     conv6 = BatchNormalization(map_rank=1)(conv6)
    conv6 = Convolution((3,3), 256, init=glorot_uniform(), activation=relu, pad=True)(conv6)
#     conv6 = BatchNormalization(map_rank=1)(conv6)
    
    up7 = C.splice(UpSampling2D(conv6), conv3, axis=0)
    conv7 = Convolution((3,3), 128, init=glorot_uniform(), activation=relu, pad=True)(up7)
#     conv7 = BatchNormalization(map_rank=1)(conv7)
    conv7 = Convolution((3,3), 128, init=glorot_uniform(), activation=relu, pad=True)(conv7)
#     conv7 = BatchNormalization(map_rank=1)(conv7)
    
    up8 = C.splice(UpSampling2D(conv7), conv2, axis=0)
    conv8 = Convolution((3,3), 64, init=glorot_uniform(), activation=relu, pad=True)(up8)
#     conv8 =BatchNormalization(map_rank=1)(conv8)
    conv8 = Convolution((3,3), 64, init=glorot_uniform(), activation=relu, pad=True)(conv8)
#     conv8 =BatchNormalization(map_rank=1)(conv8)
    
    up9 = C.splice(UpSampling2D(conv8), conv1, axis=0)
    conv9 = Convolution((3,3), 64, init=glorot_uniform(), activation=relu, pad=True)(up9)
#     conv9 = BatchNormalization(map_rank=1)(conv9)
    conv9 = Convolution((3,3), 64, init=glorot_uniform(), activation=relu, pad=True)(conv9)
#     conv9 = BatchNormalization(map_rank=1)(conv9)

    conv10 = Convolution((1,1), num_classes, init=glorot_uniform(), activation=sigmoid, pad=True)(conv9)

    return conv10

def dice_coefficient(x, y):
    # average of per-channel dice coefficient
    # global dice coefificnet doesn't work as class with larger region dominates the metrics
    # https://en.wikipedia.org/wiki/S%C3%B8rensen%E2%80%93Dice_coefficient
    intersection = C.reduce_sum(x * y, axis=(1,2))
    dice = C.reduce_mean(2.0 * intersection / (C.reduce_sum(x, axis=(1,2)) + C.reduce_sum(y, axis=(1,2)) + 1.0))
#     print("dice = " + str(dice))

    return dice

In [3]:
def slice_minibatch(all_features, all_labels, i, minibatch_size):
    batch_features = all_features[i * minibatch_size:(i + 1) * minibatch_size]
    batch_labels = all_labels[i * minibatch_size:(i + 1) * minibatch_size]

    return batch_features, batch_labels

def measure_error(data_x, data_y, x, y, trainer, minibatch_size):
    errors = []
    for i in range(0, int(len(data_x) / minibatch_size)):
        data_sx, data_sy = slice_minibatch(data_x, data_y, i, minibatch_size)

        errors.append(trainer.test_minibatch({x: data_sx, y: data_sy}))

    return np.mean(errors)

def train(images, masks, dst_folder, file_writer, train_index, test_index, num_epochs, use_existing=False):
    shape = images[0].shape
    data_size = images.shape[0]
    print(shape)
    print(data_size)

    # Split data
#     test_portion = int(data_size * 0.1)
#     indices = np.random.permutation(data_size)
#     test_indices = indices[:test_portion]
#     training_indices = indices[test_portion:]
   
    test_data = ([images[index]for index in test_index], [masks[index]for index in test_index])
    training_data = ([images[index]for index in train_index], [masks[index]for index in train_index])

    # Create model
    x = C.input_variable(shape)
    y = C.input_variable(masks[0].shape)

    z = create_model(x, masks.shape[1])
    dice_coef = dice_coefficient(z, y)

    # Load the saved model if specified
    checkpoint_file = "cntk-unet.dnn"
#     if use_existing:
#         z.load_model(checkpoint_file)

    # Prepare model and trainer
    lr = learning_rate_schedule(0.00001, UnitType.sample)
    momentum = C.learners.momentum_as_time_constant_schedule(0.9)
    trainer = C.Trainer(z, (-dice_coef, -dice_coef), C.learners.adam(z.parameters, lr=lr, momentum=momentum))

    # Get minibatches of training data and perform model training
    minibatch_size = 8
    print("train =>" + str(num_epochs))
    training_errors = []
    test_errors = []

    for e in range(0, num_epochs):
        for i in range(0, int(len(training_data[0]) / minibatch_size)):
            batch_features, batch_labels = slice_minibatch(training_data[0], training_data[1], i, minibatch_size)

            trainer.train_minibatch({x:batch_features, y: batch_labels})

        # Measure training error
        training_error = measure_error(training_data[0], training_data[1], x, y, trainer, minibatch_size)
        training_errors.append(training_error)

        # Measure test error
        test_error = measure_error(test_data[0], test_data[1], x, y, trainer, minibatch_size)
        test_errors.append(test_error)

        string_log = "\tepoch #{}: training_error={}, test_error={}".format(e, training_errors[-1], test_errors[-1])
        string_append_and_print(string_log, file_writer)
        
#         string_saved_model = "\t\tmodel saved from data set {0:.2f}".format(test_error*100 / num_minibatches_to_test)
#         string_append_and_print(string_saved_model, path_log)
        
        trainer.save_checkpoint(checkpoint_file)
#         z.save(dst_folder + "/model{0:.2f}.model".format(test_result*100 / num_minibatches_to_test))
    return trainer, training_errors, test_errors

In [4]:
input_dim_model = (3, 256, 256)
input_dim = 3 * 256 * 256
num_output_classes = 256 * 256
input = cntk.input_variable(input_dim_model)  # สังเกตว่าเราใช้ input_dim_model เป็นพารามิเตอร์แทนการใช้ input_dim
label = cntk.input_variable((256,256))


def check_folder_exist(src_folder) :
    return os.path.exists(src_folder)

def create_folder(src_folder) :
    folders = src_folder.split("/")
    current_folder = ""
    
    for folder in folders :
        current_folder += folder + "/"
        if not (folder == '' or folder == '.' or folder == '..') and not (check_folder_exist(current_folder)):
            os.mkdir(current_folder)

def create_reader_binary(path, is_training, input_dim, num_label_classes):   
#     print(path)
    labelStream = cntk.io.StreamDef(field='label', shape=num_label_classes, is_sparse=False)
    featureStream = cntk.io.StreamDef(field='features', shape=input_dim, is_sparse=False)
#     coordinateStream = cntk.io.StreamDef(field='coordinate', shape=2, is_sparse=False)
    
    deserailizer = cntk.io.CBFDeserializer(path, cntk.io.StreamDefs(label = labelStream ,feature = featureStream))
#     deserailizer = cntk.io.CTFDeserializer(path, cntk.io.StreamDefs(label = labelStream ,feature = featureStream))      
    return cntk.io.MinibatchSource(deserailizer,
       randomize = False, multithreaded_deserializer=False,max_sweeps = cntk.io.INFINITELY_REPEAT if is_training else 1)

def load_train(src) :
    src_train_file = src + "samples_train.dat"
        
    return create_reader_binary(src_train_file, True, input_dim, num_output_classes)
def string_append_and_print(line, file) :
    file_writer = open(file, 'a')
    file_writer.write(line + "\n")
    file_writer.close()
    print(line)
    

In [5]:
def get_image(src_folder, dst_folder) :
    
    samples_train_size = len(open(src_folder+"samples_train.txt","r").readlines())
    reader_train = load_train(src_folder)
    
    minibatch_size = 1
    num_samples_per_sweep = samples_train_size
    num_minibatches_to_train = (num_samples_per_sweep ) / minibatch_size

    input_map={label  : reader_train.streams.label, input  : reader_train.streams.feature} 

    print(reader_train.streams.label)
    print(reader_train.streams.feature)


    data_x, data_y = [], []
    for i in range(0, int(num_minibatches_to_train)):

        # Read a mini batch from the training data file
        data = reader_train.next_minibatch(minibatch_size, input_map=input_map) 

        training_data = []
        for k in data.keys() :
            for v in data[k].as_sequences()[0] :
                 training_data.append(v)

        data_x.append(training_data[0])
        data_y.append(training_data[1])



    labels = np.reshape(data_x, (num_samples_per_sweep,1,256,256))
    features = np.reshape(data_y,(num_samples_per_sweep,3,256,256))
    print(labels.shape)
    print(features.shape)
    
    return features, labels

In [6]:
cv = cross_validation.KFold(180, n_folds=5, shuffle=False, random_state=None)

original cnn resize to 32 x 32

train 30 epoch

In [10]:
training_src_folder = "../training_data/features/original/"
training_dst_folder = "../training_data/model/original/"


create_folder(training_dst_folder)
time = datetime.now(timezone('Asia/Bangkok')).strftime('log_date_%Y-%m-%d_time_%H-%M-%S')

path_log = training_dst_folder + time + ".txt"
file_writer = open(path_log, 'w')

training_features, training_labels = get_image(training_src_folder, training_dst_folder)
count = 1
for train_index, test_index in cv :
    round_log = "round => " + str(count) + "\n"
    train_log = "data trainng => " + str(train_index) + "\n"
    test_log = "data testing => " + str(test_index) + "\n"
    
    string_append_and_print(round_log + train_log + test_log, path_log)
    count +=1
    
    num_epochs = 30
    trainer, training_errors, test_errors = train(training_features, training_labels, training_dst_folder, path_log, train_index, test_index, num_epochs)
    
    path_model = training_dst_folder 
    name_model = "model{:.2f}".format(sum(test_errors)/len(test_errors)*100*-1)

    string_append_and_print("save model to => " + path_model + name_model + ".model", path_log)
    
    create_folder(path_model + "results/" + name_model)
    file = open( path_model + "results/" + name_model + "/training_result.txt", 'w')
    file.write(" ".join(map(str, training_errors)))
    file.close()

    file = open( path_model + "results/" + name_model + "/test_result.txt", 'w')
    file.write(" ".join(map(str, test_errors)))
    file.close()
    
    
    trainer.model.save(path_model + name_model + ".model")

label([65536])
feature([196608])
(360, 1, 256, 256)
(360, 3, 256, 256)
round => 1
data trainng => [ 36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179]
data testing => [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35]

(3, 256, 256)
360
train =>30


/home/anaconda3/envs/cntk36/lib/python3.6/site-packages/cntk/core.py:411: UserWarning: you provided the minibatch data as a list, but your corresponding input variable (uid "Input20062") has only one dynamic axis (batch axis). To speed up graph execution, please convert the data beforehand into one NumPy array to speed up training.
  'training.' % var.uid)
/home/anaconda3/envs/cntk36/lib/python3.6/site-packages/cntk/core.py:411: UserWarning: you provided the minibatch data as a list, but your corresponding input variable (uid "Input20063") has only one dynamic axis (batch axis). To speed up graph execution, please convert the data beforehand into one NumPy array to speed up training.
  'training.' % var.uid)


	epoch #0: training_error=-0.1947484388947487, test_error=-0.21280739828944206
	epoch #1: training_error=-0.2862315351764361, test_error=-0.3016459718346596
	epoch #2: training_error=-0.35486707588036853, test_error=-0.36835019290447235
	epoch #3: training_error=-0.3878182503912184, test_error=-0.4015461429953575
	epoch #4: training_error=-0.3976227483815617, test_error=-0.4117083325982094
	epoch #5: training_error=-0.3961484457055728, test_error=-0.4100511372089386
	epoch #6: training_error=-0.391210859020551, test_error=-0.4050436094403267
	epoch #7: training_error=-0.38429981221755344, test_error=-0.3982687368988991
	epoch #8: training_error=-0.3478769726223416, test_error=-0.3654297962784767
	epoch #9: training_error=-0.33216312776009244, test_error=-0.3502199873328209
	epoch #10: training_error=-0.3280553337600496, test_error=-0.34475453197956085
	epoch #11: training_error=-0.3335859427849452, test_error=-0.3490070626139641
	epoch #12: training_error=-0.3606970128085878, test_erro

/home/anaconda3/envs/cntk36/lib/python3.6/site-packages/cntk/core.py:411: UserWarning: you provided the minibatch data as a list, but your corresponding input variable (uid "Input24691") has only one dynamic axis (batch axis). To speed up graph execution, please convert the data beforehand into one NumPy array to speed up training.
  'training.' % var.uid)
/home/anaconda3/envs/cntk36/lib/python3.6/site-packages/cntk/core.py:411: UserWarning: you provided the minibatch data as a list, but your corresponding input variable (uid "Input24692") has only one dynamic axis (batch axis). To speed up graph execution, please convert the data beforehand into one NumPy array to speed up training.
  'training.' % var.uid)


	epoch #0: training_error=-0.1973818399839931, test_error=-0.16805637255311012
	epoch #1: training_error=-0.28701214989026386, test_error=-0.2501986585557461
	epoch #2: training_error=-0.36002514014641446, test_error=-0.3133910335600376
	epoch #3: training_error=-0.40004855394363403, test_error=-0.3450670391321182
	epoch #4: training_error=-0.4176131950484382, test_error=-0.3593403398990631
	epoch #5: training_error=-0.4183061470588048, test_error=-0.35890237241983414
	epoch #6: training_error=-0.4116346918874317, test_error=-0.3513198718428612
	epoch #7: training_error=-0.4038856012953652, test_error=-0.3430395722389221
	epoch #8: training_error=-0.37098490529590183, test_error=-0.31238004192709923
	epoch #9: training_error=-0.3612099654144711, test_error=-0.3037251941859722
	epoch #10: training_error=-0.35449540118376416, test_error=-0.29830365628004074
	epoch #11: training_error=-0.35042521357536316, test_error=-0.29448140040040016
	epoch #12: training_error=-0.37185973756843144, te

...

	epoch #0: training_error=-0.1933378221260177, test_error=-0.18952520564198494
	epoch #1: training_error=-0.2850714334183269, test_error=-0.27787936851382256
	epoch #2: training_error=-0.35935409367084503, test_error=-0.3551589101552963
	epoch #3: training_error=-0.39755194385846454, test_error=-0.3963603302836418
	epoch #4: training_error=-0.40800370772679645, test_error=-0.40819811075925827
	epoch #5: training_error=-0.4032788359456592, test_error=-0.40422139316797256
	epoch #6: training_error=-0.3942938297986984, test_error=-0.3948546051979065
	epoch #7: training_error=-0.3845471574200524, test_error=-0.38501299917697906
	epoch #8: training_error=-0.3745427479346593, test_error=-0.37501160800457
	epoch #9: training_error=-0.3425688048203786, test_error=-0.34230776876211166
	epoch #10: training_error=-0.32784079677528805, test_error=-0.32432085275650024
	epoch #11: training_error=-0.32558823376893997, test_error=-0.31505701690912247
	epoch #12: training_error=-0.32810232208834755, te

/home/anaconda3/envs/cntk36/lib/python3.6/site-packages/cntk/core.py:411: UserWarning: you provided the minibatch data as a list, but your corresponding input variable (uid "Input33949") has only one dynamic axis (batch axis). To speed up graph execution, please convert the data beforehand into one NumPy array to speed up training.
  'training.' % var.uid)
/home/anaconda3/envs/cntk36/lib/python3.6/site-packages/cntk/core.py:411: UserWarning: you provided the minibatch data as a list, but your corresponding input variable (uid "Input33950") has only one dynamic axis (batch axis). To speed up graph execution, please convert the data beforehand into one NumPy array to speed up training.
  'training.' % var.uid)


	epoch #0: training_error=-0.19310583670934042, test_error=-0.19605600461363792
	epoch #1: training_error=-0.2840760573744774, test_error=-0.2820149213075638
	epoch #2: training_error=-0.35958397222889793, test_error=-0.35214274004101753
	epoch #3: training_error=-0.4002338680956099, test_error=-0.39141029119491577
	epoch #4: training_error=-0.4180702832010057, test_error=-0.40911299735307693
	epoch #5: training_error=-0.42184483177132076, test_error=-0.4150615260004997
	epoch #6: training_error=-0.4185260617070728, test_error=-0.4149901866912842
	epoch #7: training_error=-0.41359438995520276, test_error=-0.4122774600982666
	epoch #8: training_error=-0.38817712830172646, test_error=-0.3891923949122429
	epoch #9: training_error=-0.36688080264462364, test_error=-0.36525576561689377
	epoch #10: training_error=-0.37649191088146633, test_error=-0.3690211996436119
	epoch #11: training_error=-0.380201088057624, test_error=-0.3650088906288147
	epoch #12: training_error=-0.39768276777532363, te

/home/anaconda3/envs/cntk36/lib/python3.6/site-packages/cntk/core.py:411: UserWarning: you provided the minibatch data as a list, but your corresponding input variable (uid "Input38578") has only one dynamic axis (batch axis). To speed up graph execution, please convert the data beforehand into one NumPy array to speed up training.
  'training.' % var.uid)
/home/anaconda3/envs/cntk36/lib/python3.6/site-packages/cntk/core.py:411: UserWarning: you provided the minibatch data as a list, but your corresponding input variable (uid "Input38579") has only one dynamic axis (batch axis). To speed up graph execution, please convert the data beforehand into one NumPy array to speed up training.
  'training.' % var.uid)


	epoch #0: training_error=-0.18724144829644096, test_error=-0.1911182776093483
	epoch #1: training_error=-0.2689485624432564, test_error=-0.2701997309923172
	epoch #2: training_error=-0.34397078222698635, test_error=-0.3414178341627121
	epoch #3: training_error=-0.3875889546341366, test_error=-0.38376274704933167
	epoch #4: training_error=-0.4073924885855781, test_error=-0.4082988426089287
	epoch #5: training_error=-0.40838803847630817, test_error=-0.41182637214660645
	epoch #6: training_error=-0.40291223261091447, test_error=-0.40656983852386475
	epoch #7: training_error=-0.39364273846149445, test_error=-0.3984096720814705
	epoch #8: training_error=-0.37319047417905593, test_error=-0.38041429221630096
	epoch #9: training_error=-0.34395571715301937, test_error=-0.35465332120656967
	epoch #10: training_error=-0.3373313744862874, test_error=-0.3476850539445877
	epoch #11: training_error=-0.3340894381205241, test_error=-0.3415831923484802
	epoch #12: training_error=-0.3310212261146969, te

train 100 epoch

In [26]:
training_src_folder = "../training_data/features/original/"
training_dst_folder = "../training_data/model/original/"


create_folder(training_dst_folder)
time = datetime.now(timezone('Asia/Bangkok')).strftime('log_date_%Y-%m-%d_time_%H-%M-%S')

path_log = training_dst_folder + time + ".txt"
file_writer = open(path_log, 'w')

training_features, training_labels = get_image(training_src_folder, training_dst_folder)
count = 1
for train_index, test_index in cv :
    round_log = "round => " + str(count) + "\n"
    train_log = "data trainng => " + str(train_index) + "\n"
    test_log = "data testing => " + str(test_index) + "\n"
    
    string_append_and_print(round_log + train_log + test_log, path_log)
    count +=1
    
    num_epochs = 100
    trainer, training_errors, test_errors = train(training_features, training_labels, training_dst_folder, path_log, train_index, test_index, num_epochs)
    
    path_model = training_dst_folder 
    name_model = "model{:.2f}".format(sum(test_errors)/len(test_errors)*100*-1)

    string_append_and_print("save model to => " + path_model + name_model + ".model", path_log)
    
    create_folder(path_model + "results/" + name_model)
    file = open( path_model + "results/" + name_model + "/training_result.txt", 'w')
    file.write(" ".join(map(str, training_errors)))
    file.close()

    file = open( path_model + "results/" + name_model + "/test_result.txt", 'w')
    file.write(" ".join(map(str, test_errors)))
    file.close()
    
    
    trainer.model.save(path_model + name_model + ".model")

label([65536])
feature([196608])
(360, 1, 256, 256)
(360, 3, 256, 256)
round => 1
data trainng => [ 36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179]
data testing => [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35]

(3, 256, 256)
360
train =>100


/home/anaconda3/envs/cntk36/lib/python3.6/site-packages/cntk/core.py:411: UserWarning: you provided the minibatch data as a list, but your corresponding input variable (uid "Input223842") has only one dynamic axis (batch axis). To speed up graph execution, please convert the data beforehand into one NumPy array to speed up training.
  'training.' % var.uid)
/home/anaconda3/envs/cntk36/lib/python3.6/site-packages/cntk/core.py:411: UserWarning: you provided the minibatch data as a list, but your corresponding input variable (uid "Input223843") has only one dynamic axis (batch axis). To speed up graph execution, please convert the data beforehand into one NumPy array to speed up training.
  'training.' % var.uid)


	epoch #0: training_error=-0.19439317244622442, test_error=-0.20616347715258598
	epoch #1: training_error=-0.26103125512599945, test_error=-0.24956636130809784
	epoch #2: training_error=-0.3360243754254447, test_error=-0.3145151697099209
	epoch #3: training_error=-0.37581681791279054, test_error=-0.3560727462172508
	epoch #4: training_error=-0.38783322936958736, test_error=-0.3585760146379471
	epoch #5: training_error=-0.31601818030079204, test_error=-0.23893882147967815
	epoch #6: training_error=-0.36278725746605134, test_error=-0.2995632514357567
	epoch #7: training_error=-0.4309922473298179, test_error=-0.3973687142133713
	epoch #8: training_error=-0.2714479685657554, test_error=-0.18371557630598545
	epoch #9: training_error=-0.3765563815832138, test_error=-0.29084382206201553
	epoch #10: training_error=-0.44365914828247494, test_error=-0.3758745416998863
	epoch #11: training_error=-0.3268824840585391, test_error=-0.23190593905746937
	epoch #12: training_error=-0.3064675447013643, t

/home/anaconda3/envs/cntk36/lib/python3.6/site-packages/cntk/core.py:411: UserWarning: you provided the minibatch data as a list, but your corresponding input variable (uid "Input226993") has only one dynamic axis (batch axis). To speed up graph execution, please convert the data beforehand into one NumPy array to speed up training.
  'training.' % var.uid)
/home/anaconda3/envs/cntk36/lib/python3.6/site-packages/cntk/core.py:411: UserWarning: you provided the minibatch data as a list, but your corresponding input variable (uid "Input226994") has only one dynamic axis (batch axis). To speed up graph execution, please convert the data beforehand into one NumPy array to speed up training.
  'training.' % var.uid)


	epoch #0: training_error=-0.19820590565601984, test_error=-0.17101153172552586
	epoch #1: training_error=-0.25291261821985245, test_error=-0.23547141253948212
	epoch #2: training_error=-0.32335567226012546, test_error=-0.31519998982548714
	epoch #3: training_error=-0.36478227294153637, test_error=-0.33970581740140915
	epoch #4: training_error=-0.38012447622087264, test_error=-0.35062630474567413
	epoch #5: training_error=-0.40024081865946454, test_error=-0.37415359914302826
	epoch #6: training_error=-0.4040280630191167, test_error=-0.3904438465833664
	epoch #7: training_error=-0.3227127633161015, test_error=-0.38602447509765625
	epoch #8: training_error=-0.3194880435864131, test_error=-0.3866204619407654
	epoch #9: training_error=-0.37409981588522595, test_error=-0.4242415502667427
	epoch #10: training_error=-0.3660212490293715, test_error=-0.4184849262237549
	epoch #11: training_error=-0.25122845504019, test_error=-0.3088319469243288
	epoch #12: training_error=-0.3253364496760898, te

...

/home/anaconda3/envs/cntk36/lib/python3.6/site-packages/cntk/core.py:411: UserWarning: you provided the minibatch data as a list, but your corresponding input variable (uid "Input233295") has only one dynamic axis (batch axis). To speed up graph execution, please convert the data beforehand into one NumPy array to speed up training.
  'training.' % var.uid)
/home/anaconda3/envs/cntk36/lib/python3.6/site-packages/cntk/core.py:411: UserWarning: you provided the minibatch data as a list, but your corresponding input variable (uid "Input233296") has only one dynamic axis (batch axis). To speed up graph execution, please convert the data beforehand into one NumPy array to speed up training.
  'training.' % var.uid)


	epoch #0: training_error=-0.19858932991822562, test_error=-0.19427024945616722
	epoch #1: training_error=-0.2656390716632207, test_error=-0.24440718814730644
	epoch #2: training_error=-0.33285840186807847, test_error=-0.30316001549363136
	epoch #3: training_error=-0.3731665205624368, test_error=-0.35947590321302414
	epoch #4: training_error=-0.37759435921907425, test_error=-0.35808445513248444
	epoch #5: training_error=-0.4184464224510723, test_error=-0.41749299317598343
	epoch #6: training_error=-0.4376449336608251, test_error=-0.43072502315044403
	epoch #7: training_error=-0.35760610964563155, test_error=-0.33713047951459885
	epoch #8: training_error=-0.4327334463596344, test_error=-0.4332195743918419
	epoch #9: training_error=-0.4625355617867576, test_error=-0.46231964975595474
	epoch #10: training_error=-0.47779637409581077, test_error=-0.4361994117498398
	epoch #11: training_error=-0.4853276213010152, test_error=-0.42371125519275665
	epoch #12: training_error=-0.5095879369311862,

train =>100


/home/anaconda3/envs/cntk36/lib/python3.6/site-packages/cntk/core.py:411: UserWarning: you provided the minibatch data as a list, but your corresponding input variable (uid "Input236446") has only one dynamic axis (batch axis). To speed up graph execution, please convert the data beforehand into one NumPy array to speed up training.
  'training.' % var.uid)
/home/anaconda3/envs/cntk36/lib/python3.6/site-packages/cntk/core.py:411: UserWarning: you provided the minibatch data as a list, but your corresponding input variable (uid "Input236447") has only one dynamic axis (batch axis). To speed up graph execution, please convert the data beforehand into one NumPy array to speed up training.
  'training.' % var.uid)


	epoch #0: training_error=-0.18953395883242288, test_error=-0.19879337400197983
	epoch #1: training_error=-0.25659338053729797, test_error=-0.2687397822737694
	epoch #2: training_error=-0.34023943129513, test_error=-0.3378262296319008
	epoch #3: training_error=-0.3355429578158591, test_error=-0.3373688533902168
	epoch #4: training_error=-0.3811037043730418, test_error=-0.36414726451039314
	epoch #5: training_error=-0.4155417639348242, test_error=-0.3877374902367592
	epoch #6: training_error=-0.43180956112013924, test_error=-0.4014100804924965
	epoch #7: training_error=-0.44432318210601807, test_error=-0.41583722829818726
	epoch #8: training_error=-0.4266651173432668, test_error=-0.4223654791712761
	epoch #9: training_error=-0.419843057791392, test_error=-0.4222167879343033
	epoch #10: training_error=-0.45127160681618583, test_error=-0.442602202296257
	epoch #11: training_error=-0.45634501841333175, test_error=-0.44641467928886414
	epoch #12: training_error=-0.40971527331405216, test_er

train 200 epoch

In [19]:
training_src_folder = "../training_data/features/original/"
training_dst_folder = "../training_data/model/original/"


create_folder(training_dst_folder)
time = datetime.now(timezone('Asia/Bangkok')).strftime('log_date_%Y-%m-%d_time_%H-%M-%S')

path_log = training_dst_folder + time + ".txt"
file_writer = open(path_log, 'w')

training_features, training_labels = get_image(training_src_folder, training_dst_folder)
count = 1
for train_index, test_index in cv :
    round_log = "round => " + str(count) + "\n"
    train_log = "data trainng => " + str(train_index) + "\n"
    test_log = "data testing => " + str(test_index) + "\n"
    
    string_append_and_print(round_log + train_log + test_log, path_log)
    count +=1
    
    num_epochs = 200
    trainer, training_errors, test_errors = train(training_features, training_labels, training_dst_folder, path_log, train_index, test_index, num_epochs)
    
    path_model = training_dst_folder 
    name_model = "model{:.2f}".format(sum(test_errors)/len(test_errors)*100*-1)

    string_append_and_print("save model to => " + path_model + name_model + ".model", path_log)
    
    create_folder(path_model + "results/" + name_model)
    file = open( path_model + "results/" + name_model + "/training_result.txt", 'w')
    file.write(" ".join(map(str, training_errors)))
    file.close()

    file = open( path_model + "results/" + name_model + "/test_result.txt", 'w')
    file.write(" ".join(map(str, test_errors)))
    file.close()
    
    
    trainer.model.save(path_model + name_model + ".model")

label([65536])
feature([196608])
(360, 1, 256, 256)
(360, 3, 256, 256)
round => 1
data trainng => [ 36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179]
data testing => [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35]

(3, 256, 256)
360
train =>200


/home/anaconda3/envs/cntk36/lib/python3.6/site-packages/cntk/core.py:411: UserWarning: you provided the minibatch data as a list, but your corresponding input variable (uid "Input138572") has only one dynamic axis (batch axis). To speed up graph execution, please convert the data beforehand into one NumPy array to speed up training.
  'training.' % var.uid)
/home/anaconda3/envs/cntk36/lib/python3.6/site-packages/cntk/core.py:411: UserWarning: you provided the minibatch data as a list, but your corresponding input variable (uid "Input138573") has only one dynamic axis (batch axis). To speed up graph execution, please convert the data beforehand into one NumPy array to speed up training.
  'training.' % var.uid)


	epoch #0: training_error=-0.16194275932179558, test_error=-0.16600144654512405
	epoch #1: training_error=-0.3512157276272774, test_error=-0.36939989775419235
	epoch #2: training_error=-0.5514367421468099, test_error=-0.5562711656093597
	epoch #3: training_error=-0.5987763106822968, test_error=-0.5992738902568817
	epoch #4: training_error=-0.6368484795093536, test_error=-0.6260240375995636
	epoch #5: training_error=-0.6575984872049756, test_error=-0.6472662687301636
	epoch #6: training_error=-0.6706079973114861, test_error=-0.660996213555336
	epoch #7: training_error=-0.6762430253956053, test_error=-0.6657821536064148
	epoch #8: training_error=-0.6844935251606835, test_error=-0.6704147011041641
	epoch #9: training_error=-0.6932378030485578, test_error=-0.6755014657974243
	epoch #10: training_error=-0.6961505495839648, test_error=-0.6770072728395462
	epoch #11: training_error=-0.6987228956487443, test_error=-0.6807586997747421
	epoch #12: training_error=-0.6999829494290881, test_error=-

	epoch #105: training_error=-0.8886536293559604, test_error=-0.7927414178848267
	epoch #106: training_error=-0.9005855520566305, test_error=-0.8017822951078415
	epoch #107: training_error=-0.9069151547220018, test_error=-0.8071648478507996
	epoch #108: training_error=-0.8967505428526137, test_error=-0.8005788028240204
	epoch #109: training_error=-0.9016833901405334, test_error=-0.8034291416406631
	epoch #110: training_error=-0.8990465501944224, test_error=-0.7989656925201416
	epoch #111: training_error=-0.8938311139742533, test_error=-0.7970341742038727
	epoch #112: training_error=-0.9050034681955973, test_error=-0.802753135561943
	epoch #113: training_error=-0.9159983164734311, test_error=-0.8130867183208466
	epoch #114: training_error=-0.8906249933772616, test_error=-0.7924131900072098
	epoch #115: training_error=-0.9173987905184428, test_error=-0.8182534575462341
	epoch #116: training_error=-0.8928902414109972, test_error=-0.8008161187171936
	epoch #117: training_error=-0.9121873511

train =>200


...

	epoch #105: training_error=-0.9421858423286014, test_error=-0.769083634018898
	epoch #106: training_error=-0.9414548608991835, test_error=-0.7643850743770599
	epoch #107: training_error=-0.9253110455142127, test_error=-0.76235631108284
	epoch #108: training_error=-0.9117306305302514, test_error=-0.7532225251197815
	epoch #109: training_error=-0.9341043631235758, test_error=-0.7573173940181732
	epoch #110: training_error=-0.9320079055097368, test_error=-0.7662715464830399
	epoch #111: training_error=-0.9387151267793443, test_error=-0.7659459710121155
	epoch #112: training_error=-0.9463686645030975, test_error=-0.771391436457634
	epoch #113: training_error=-0.9245321055253347, test_error=-0.7612393647432327
	epoch #114: training_error=-0.9450820055272844, test_error=-0.7674988806247711
	epoch #115: training_error=-0.9371238682005141, test_error=-0.7666840255260468
	epoch #116: training_error=-0.9475323855876923, test_error=-0.7687320113182068
	epoch #117: training_error=-0.9452660547362

train =>200


/home/anaconda3/envs/cntk36/lib/python3.6/site-packages/cntk/core.py:411: UserWarning: you provided the minibatch data as a list, but your corresponding input variable (uid "Input152444") has only one dynamic axis (batch axis). To speed up graph execution, please convert the data beforehand into one NumPy array to speed up training.
  'training.' % var.uid)
/home/anaconda3/envs/cntk36/lib/python3.6/site-packages/cntk/core.py:411: UserWarning: you provided the minibatch data as a list, but your corresponding input variable (uid "Input152445") has only one dynamic axis (batch axis). To speed up graph execution, please convert the data beforehand into one NumPy array to speed up training.
  'training.' % var.uid)


	epoch #0: training_error=-0.15444169814387956, test_error=-0.16268014907836914
	epoch #1: training_error=-0.3536670207977295, test_error=-0.36130592226982117
	epoch #2: training_error=-0.47733383377393085, test_error=-0.408508762717247
	epoch #3: training_error=-0.5244013203514947, test_error=-0.42246561497449875
	epoch #4: training_error=-0.48991624183124965, test_error=-0.3780646547675133
	epoch #5: training_error=-0.44409066604243386, test_error=-0.3377259187400341
	epoch #6: training_error=-0.4882892088757621, test_error=-0.37175483256578445
	epoch #7: training_error=-0.3946398024757703, test_error=-0.29393429681658745
	epoch #8: training_error=-0.2740130134754711, test_error=-0.1909189634025097
	epoch #9: training_error=-0.34832797530624604, test_error=-0.22975853085517883
	epoch #10: training_error=-0.40028732766707736, test_error=-0.29813016951084137
	epoch #11: training_error=-0.4055190318160587, test_error=-0.2737724892795086
	epoch #12: training_error=-0.2916860166523192, te

	epoch #104: training_error=-0.9004762040244209, test_error=-0.7757282555103302
	epoch #105: training_error=-0.9259486397107443, test_error=-0.7345788180828094
	epoch #106: training_error=-0.9218704667356279, test_error=-0.7828505337238312
	epoch #107: training_error=-0.8826497395833334, test_error=-0.7561780661344528
	epoch #108: training_error=-0.9270008934868706, test_error=-0.7785015255212784
	epoch #109: training_error=-0.8823543687661489, test_error=-0.7477566003799438
	epoch #110: training_error=-0.9306968649228414, test_error=-0.7751104235649109
	epoch #111: training_error=-0.927888360288408, test_error=-0.6842631101608276
	epoch #112: training_error=-0.8220530947049459, test_error=-0.7215986996889114
	epoch #113: training_error=-0.9348482853836484, test_error=-0.7176501750946045
	epoch #114: training_error=-0.8931611378987631, test_error=-0.7708441019058228
	epoch #115: training_error=-0.9293483164575365, test_error=-0.7137888371944427
	epoch #116: training_error=-0.9120445781

train 300 epoch

In [24]:
training_src_folder = "../training_data/features/original/"
training_dst_folder = "../training_data/model/original/"


create_folder(training_dst_folder)
time = datetime.now(timezone('Asia/Bangkok')).strftime('log_date_%Y-%m-%d_time_%H-%M-%S')

path_log = training_dst_folder + time + ".txt"
file_writer = open(path_log, 'w')

training_features, training_labels = get_image(training_src_folder, training_dst_folder)
count = 1
for train_index, test_index in cv :
    round_log = "round => " + str(count) + "\n"
    train_log = "data trainng => " + str(train_index) + "\n"
    test_log = "data testing => " + str(test_index) + "\n"
    
    string_append_and_print(round_log + train_log + test_log, path_log)
    count +=1
    
    num_epochs = 300
    trainer, training_errors, test_errors = train(training_features, training_labels, training_dst_folder, path_log, train_index, test_index, num_epochs)
    
    path_model = training_dst_folder 
    name_model = "model{:.2f}".format(sum(test_errors)/len(test_errors)*100*-1)

    string_append_and_print("save model to => " + path_model + name_model + ".model", path_log)
    
    create_folder(path_model + "results/" + name_model)
    file = open( path_model + "results/" + name_model + "/training_result.txt", 'w')
    file.write(" ".join(map(str, training_errors)))
    file.close()

    file = open( path_model + "results/" + name_model + "/test_result.txt", 'w')
    file.write(" ".join(map(str, test_errors)))
    file.close()
    
    
    trainer.model.save(path_model + name_model + ".model")

label([65536])
feature([196608])
(360, 1, 256, 256)
(360, 3, 256, 256)
round => 1
data trainng => [ 36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179]
data testing => [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35]

(3, 256, 256)
360
train =>300


/home/anaconda3/envs/cntk36/lib/python3.6/site-packages/cntk/core.py:411: UserWarning: you provided the minibatch data as a list, but your corresponding input variable (uid "Input207217") has only one dynamic axis (batch axis). To speed up graph execution, please convert the data beforehand into one NumPy array to speed up training.
  'training.' % var.uid)
/home/anaconda3/envs/cntk36/lib/python3.6/site-packages/cntk/core.py:411: UserWarning: you provided the minibatch data as a list, but your corresponding input variable (uid "Input207218") has only one dynamic axis (batch axis). To speed up graph execution, please convert the data beforehand into one NumPy array to speed up training.
  'training.' % var.uid)


	epoch #0: training_error=-0.20566471500529182, test_error=-0.21155470237135887
	epoch #1: training_error=-0.2760279592540529, test_error=-0.24964062124490738
	epoch #2: training_error=-0.3122767408688863, test_error=-0.2735845670104027
	epoch #3: training_error=-0.31862328863806194, test_error=-0.2627154216170311
	epoch #4: training_error=-0.15424925264798933, test_error=-0.07118422433268279
	epoch #5: training_error=-0.19207194737262195, test_error=-0.09712626971304417
	epoch #6: training_error=-0.20716184998552004, test_error=-0.1268579065799713
	epoch #7: training_error=-0.17225096291965908, test_error=-0.08050367137184367
	epoch #8: training_error=-0.16372091157568824, test_error=-0.0703042657696642
	epoch #9: training_error=-0.20232931048505837, test_error=-0.10176519816741347
	epoch #10: training_error=-0.15553976160784563, test_error=-0.07237657846417278
	epoch #11: training_error=-0.18715394764310783, test_error=-0.09781319973990321
	epoch #12: training_error=-0.24167645060353

	epoch #103: training_error=-0.5781495620807012, test_error=-0.3435062915086746
	epoch #104: training_error=-0.49240032500690883, test_error=-0.3191140927374363
	epoch #105: training_error=-0.48223984158701366, test_error=-0.30100215040147305
	epoch #106: training_error=-0.49745838675234055, test_error=-0.31872253119945526
	epoch #107: training_error=-0.4190123975276947, test_error=-0.2711865659803152
	epoch #108: training_error=-0.47920669780837166, test_error=-0.2846139073371887
	epoch #109: training_error=-0.4593939118915134, test_error=-0.2834248747676611
	epoch #110: training_error=-0.444900769326422, test_error=-0.28577212803065777
	epoch #111: training_error=-0.5691086649894714, test_error=-0.3357982374727726
	epoch #112: training_error=-0.48461860087182784, test_error=-0.3057498335838318
	epoch #113: training_error=-0.4738069623708725, test_error=-0.30304646864533424
	epoch #114: training_error=-0.5168389628330866, test_error=-0.3107525147497654
	epoch #115: training_error=-0.4

	epoch #206: training_error=-0.8675352070066664, test_error=-0.34256407618522644
	epoch #207: training_error=-0.8505229718155332, test_error=-0.34041551500558853
	epoch #208: training_error=-0.8606284360090891, test_error=-0.3314908817410469
	epoch #209: training_error=-0.773743189043469, test_error=-0.32098687067627907
	epoch #210: training_error=-0.8752032452159457, test_error=-0.35516566783189774
	epoch #211: training_error=-0.8043389552169375, test_error=-0.3248925879597664
	epoch #212: training_error=-0.8698732654253641, test_error=-0.3546912968158722
	epoch #213: training_error=-0.6801748093631532, test_error=-0.30528031662106514
	epoch #214: training_error=-0.8730133970578512, test_error=-0.3404736816883087
	epoch #215: training_error=-0.8605318499935998, test_error=-0.3360728919506073
	epoch #216: training_error=-0.8588397536012862, test_error=-0.32733481377363205
	epoch #217: training_error=-0.8796199560165405, test_error=-0.3496594503521919
	epoch #218: training_error=-0.8821

...

train =>300


/home/anaconda3/envs/cntk36/lib/python3.6/site-packages/cntk/core.py:411: UserWarning: you provided the minibatch data as a list, but your corresponding input variable (uid "Input220517") has only one dynamic axis (batch axis). To speed up graph execution, please convert the data beforehand into one NumPy array to speed up training.
  'training.' % var.uid)
/home/anaconda3/envs/cntk36/lib/python3.6/site-packages/cntk/core.py:411: UserWarning: you provided the minibatch data as a list, but your corresponding input variable (uid "Input220518") has only one dynamic axis (batch axis). To speed up graph execution, please convert the data beforehand into one NumPy array to speed up training.
  'training.' % var.uid)


	epoch #0: training_error=-0.20389269126786125, test_error=-0.2175556719303131
	epoch #1: training_error=-0.2708939015865326, test_error=-0.29216910898685455
	epoch #2: training_error=-0.25022586186726886, test_error=-0.27825306355953217
	epoch #3: training_error=-0.2882757700151867, test_error=-0.3050529435276985
	epoch #4: training_error=-0.2010643697447247, test_error=-0.21740965731441975
	epoch #5: training_error=-0.148155324988895, test_error=-0.1607216913253069
	epoch #6: training_error=-0.19219578471448687, test_error=-0.18852228671312332
	epoch #7: training_error=-0.15618174502419102, test_error=-0.1675543300807476
	epoch #8: training_error=-0.14537700327734152, test_error=-0.14447756670415401
	epoch #9: training_error=-0.08923497415768604, test_error=-0.11181072238832712
	epoch #10: training_error=-0.04366421923300044, test_error=-0.06210252596065402
	epoch #11: training_error=-0.05243754778995127, test_error=-0.06934359949082136
	epoch #12: training_error=-0.09356146481715971

	epoch #103: training_error=-0.3663542725973659, test_error=-0.19463306479156017
	epoch #104: training_error=-0.4159611099296146, test_error=-0.22660833038389683
	epoch #105: training_error=-0.41768981516361237, test_error=-0.22460472583770752
	epoch #106: training_error=-0.4941673345035977, test_error=-0.2651075981557369
	epoch #107: training_error=-0.40331338180436027, test_error=-0.21384787745773792
	epoch #108: training_error=-0.4020903830726941, test_error=-0.22866827249526978
	epoch #109: training_error=-0.47318874961800045, test_error=-0.257736062631011
	epoch #110: training_error=-0.4064093406001727, test_error=-0.20892044715583324
	epoch #111: training_error=-0.48608602583408356, test_error=-0.24964487366378307
	epoch #112: training_error=-0.32870033217800987, test_error=-0.18129554577171803
	epoch #113: training_error=-0.5242275595664978, test_error=-0.2856791652739048
	epoch #114: training_error=-0.39122508549027973, test_error=-0.2001203838735819
	epoch #115: training_error

	epoch #205: training_error=-0.7674454847971598, test_error=-0.26766774989664555
	epoch #206: training_error=-0.8250679075717926, test_error=-0.30394933745265007
	epoch #207: training_error=-0.8089842630757226, test_error=-0.28865572810173035
	epoch #208: training_error=-0.8616806003782485, test_error=-0.35398944839835167
	epoch #209: training_error=-0.8410210874345567, test_error=-0.31512733176350594
	epoch #210: training_error=-0.8064785103003184, test_error=-0.2828303426504135
	epoch #211: training_error=-0.8595027459992303, test_error=-0.32954055815935135
	epoch #212: training_error=-0.8599551154507531, test_error=-0.33300524204969406
	epoch #213: training_error=-0.8638014958964454, test_error=-0.32732612267136574
	epoch #214: training_error=-0.8199295136663649, test_error=-0.2947870343923569
	epoch #215: training_error=-0.8414298031065199, test_error=-0.29782911762595177
	epoch #216: training_error=-0.870738748047087, test_error=-0.3675980791449547
	epoch #217: training_error=-0.8

train 500 epoch

In [ ]:
training_src_folder = "../training_data/features/original/"
training_dst_folder = "../training_data/model/original/"


create_folder(training_dst_folder)
time = datetime.now(timezone('Asia/Bangkok')).strftime('log_date_%Y-%m-%d_time_%H-%M-%S')

path_log = training_dst_folder + time + ".txt"
file_writer = open(path_log, 'w')

training_features, training_labels = get_image(training_src_folder, training_dst_folder)
count = 1
for train_index, test_index in cv :
    round_log = "round => " + str(count) + "\n"
    train_log = "data trainng => " + str(train_index) + "\n"
    test_log = "data testing => " + str(test_index) + "\n"
    
    string_append_and_print(round_log + train_log + test_log, path_log)
    count +=1
    
    num_epochs = 500
    trainer, training_errors, test_errors = train(training_features, training_labels, training_dst_folder, path_log, train_index, test_index, num_epochs)
    
    path_model = training_dst_folder 
    name_model = "model{:.2f}".format(sum(test_errors)/len(test_errors)*100*-1)

    string_append_and_print("save model to => " + path_model + name_model + ".model", path_log)
    
    create_folder(path_model + "results/" + name_model)
    file = open( path_model + "results/" + name_model + "/training_result.txt", 'w')
    file.write(" ".join(map(str, training_errors)))
    file.close()

    file = open( path_model + "results/" + name_model + "/test_result.txt", 'w')
    file.write(" ".join(map(str, test_errors)))
    file.close()
    
    
    trainer.model.save(path_model + name_model + ".model")

label([65536])
feature([196608])
(360, 1, 256, 256)
(360, 3, 256, 256)
round => 1
data trainng => [ 36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179]
data testing => [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35]

(3, 256, 256)
360
train =>500


/home/anaconda3/envs/cntk36/lib/python3.6/site-packages/cntk/core.py:411: UserWarning: you provided the minibatch data as a list, but your corresponding input variable (uid "Input173252") has only one dynamic axis (batch axis). To speed up graph execution, please convert the data beforehand into one NumPy array to speed up training.
  'training.' % var.uid)
/home/anaconda3/envs/cntk36/lib/python3.6/site-packages/cntk/core.py:411: UserWarning: you provided the minibatch data as a list, but your corresponding input variable (uid "Input173253") has only one dynamic axis (batch axis). To speed up graph execution, please convert the data beforehand into one NumPy array to speed up training.
  'training.' % var.uid)


	epoch #0: training_error=-0.16202568842305076, test_error=-0.1661273092031479
	epoch #1: training_error=-0.35191645307673347, test_error=-0.370352603495121
	epoch #2: training_error=-0.5545796503623327, test_error=-0.5585877373814583
	epoch #3: training_error=-0.6007583091656367, test_error=-0.6006817519664764
	epoch #4: training_error=-0.638382246096929, test_error=-0.627010852098465
	epoch #5: training_error=-0.6588654849264357, test_error=-0.6480816155672073
	epoch #6: training_error=-0.670641667313046, test_error=-0.6615060418844223
	epoch #7: training_error=-0.6769728031423357, test_error=-0.6674181520938873
	epoch #8: training_error=-0.6854407257503934, test_error=-0.6718361675739288
	epoch #9: training_error=-0.6936715857850181, test_error=-0.6774055510759354
	epoch #10: training_error=-0.6965633382399877, test_error=-0.6760643571615219
	epoch #11: training_error=-0.698643672797415, test_error=-0.6810631603002548
	epoch #12: training_error=-0.6978238970041275, test_error=-0.672

	epoch #105: training_error=-0.9167643752362993, test_error=-0.8242982476949692
	epoch #106: training_error=-0.8540062076515622, test_error=-0.7742414623498917
	epoch #107: training_error=-0.8942641152275933, test_error=-0.7988136559724808
	epoch #108: training_error=-0.8904168638918135, test_error=-0.7959192842245102
	epoch #109: training_error=-0.8971388571792178, test_error=-0.8042667359113693
	epoch #110: training_error=-0.9229629834493002, test_error=-0.8259266763925552
	epoch #111: training_error=-0.8944623238510556, test_error=-0.7992884069681168
	epoch #112: training_error=-0.9093322621451484, test_error=-0.8141671419143677
	epoch #113: training_error=-0.9067943261729347, test_error=-0.8132420629262924
	epoch #114: training_error=-0.9103867279158698, test_error=-0.816064178943634
	epoch #115: training_error=-0.9233236047956679, test_error=-0.8219055533409119
	epoch #116: training_error=-0.8875199622578092, test_error=-0.7960301339626312
	epoch #117: training_error=-0.9215356806

	epoch #208: training_error=-0.9525323642624749, test_error=-0.8471156060695648
	epoch #209: training_error=-0.941212750143475, test_error=-0.8333585262298584
	epoch #210: training_error=-0.9517888161871169, test_error=-0.8460413664579391
	epoch #211: training_error=-0.9453072216775682, test_error=-0.8356879502534866
	epoch #212: training_error=-0.9470726450284322, test_error=-0.8450041562318802
	epoch #213: training_error=-0.9496660364998711, test_error=-0.844638466835022
	epoch #214: training_error=-0.9486650625864664, test_error=-0.8455799967050552
	epoch #215: training_error=-0.9484841227531433, test_error=-0.8400862812995911
	epoch #216: training_error=-0.9456546439064873, test_error=-0.8379060626029968
	epoch #217: training_error=-0.9524832533465492, test_error=-0.8451272398233414
	epoch #218: training_error=-0.950388663344913, test_error=-0.842939630150795
	epoch #219: training_error=-0.9494199554125468, test_error=-0.8452675342559814
	epoch #220: training_error=-0.9474756717681

...

/home/anaconda3/envs/cntk36/lib/python3.6/site-packages/cntk/core.py:411: UserWarning: you provided the minibatch data as a list, but your corresponding input variable (uid "Input183656") has only one dynamic axis (batch axis). To speed up graph execution, please convert the data beforehand into one NumPy array to speed up training.
  'training.' % var.uid)
/home/anaconda3/envs/cntk36/lib/python3.6/site-packages/cntk/core.py:411: UserWarning: you provided the minibatch data as a list, but your corresponding input variable (uid "Input183657") has only one dynamic axis (batch axis). To speed up graph execution, please convert the data beforehand into one NumPy array to speed up training.
  'training.' % var.uid)


train =>500
	epoch #0: training_error=-0.16057189347015488, test_error=-0.16532305628061295
	epoch #1: training_error=-0.347354037894143, test_error=-0.35726049542427063
	epoch #2: training_error=-0.5428889559374915, test_error=-0.5306982845067978
	epoch #3: training_error=-0.6269533601072099, test_error=-0.5832733660936356
	epoch #4: training_error=-0.657049255238639, test_error=-0.5997073277831078
	epoch #5: training_error=-0.667653684814771, test_error=-0.6021322906017303
	epoch #6: training_error=-0.6776174853245417, test_error=-0.6079287454485893
	epoch #7: training_error=-0.6892788973119524, test_error=-0.6074537038803101
	epoch #8: training_error=-0.6931547919909159, test_error=-0.6064614504575729
	epoch #9: training_error=-0.7064866258038415, test_error=-0.6213911771774292
	epoch #10: training_error=-0.7158010966247983, test_error=-0.5927199348807335
	epoch #11: training_error=-0.7385139034854041, test_error=-0.646342821419239
	epoch #12: training_error=-0.7571679949760437, tes

	epoch #104: training_error=-0.92170583208402, test_error=-0.7716342508792877
	epoch #105: training_error=-0.9189631905820634, test_error=-0.760757103562355
	epoch #106: training_error=-0.9399838348229727, test_error=-0.7678676694631577
	epoch #107: training_error=-0.9249402085940043, test_error=-0.7632463872432709
	epoch #108: training_error=-0.9362304707368215, test_error=-0.7630437910556793
	epoch #109: training_error=-0.9441364606221517, test_error=-0.7687178701162338
	epoch #110: training_error=-0.9418068726857504, test_error=-0.7662581205368042
	epoch #111: training_error=-0.9179978668689728, test_error=-0.7499729245901108
	epoch #112: training_error=-0.9436250163449181, test_error=-0.7650089114904404
	epoch #113: training_error=-0.9282119737731086, test_error=-0.7627720683813095
	epoch #114: training_error=-0.9429172409905328, test_error=-0.7675055712461472
	epoch #115: training_error=-0.9400237103303274, test_error=-0.7713185399770737
	epoch #116: training_error=-0.946179168091

	epoch #207: training_error=-0.9624678393205007, test_error=-0.7693111002445221
	epoch #208: training_error=-0.962027241786321, test_error=-0.7703219205141068
	epoch #209: training_error=-0.9600201083554162, test_error=-0.7640637010335922
	epoch #210: training_error=-0.9579506814479828, test_error=-0.7641697824001312
	epoch #211: training_error=-0.9616201255056593, test_error=-0.7633098810911179
	epoch #212: training_error=-0.9582507775889503, test_error=-0.7659282982349396
	epoch #213: training_error=-0.9582631091276804, test_error=-0.7620509713888168
	epoch #214: training_error=-0.9624875750806596, test_error=-0.7722697407007217
	epoch #215: training_error=-0.9635058641433716, test_error=-0.7673279047012329
	epoch #216: training_error=-0.9527739650673337, test_error=-0.7597906738519669
	epoch #217: training_error=-0.9632491966088613, test_error=-0.7674137502908707
	epoch #218: training_error=-0.9497298730744256, test_error=-0.7583383172750473
	epoch #219: training_error=-0.9632057183

	epoch #310: training_error=-0.9564117524358962, test_error=-0.7643317580223083
	epoch #311: training_error=-0.9623291591803232, test_error=-0.7632529735565186
	epoch #312: training_error=-0.9641697936587863, test_error=-0.7636886984109879
	epoch #313: training_error=-0.9557622704241011, test_error=-0.7619998157024384
	epoch #314: training_error=-0.9616264137956831, test_error=-0.7663211971521378
	epoch #315: training_error=-0.9626324474811554, test_error=-0.7634388953447342
	epoch #316: training_error=-0.9656056728627946, test_error=-0.7702805697917938
	epoch #317: training_error=-0.9548277656237284, test_error=-0.7595408856868744
	epoch #318: training_error=-0.9616194334295061, test_error=-0.7601429373025894
	epoch #319: training_error=-0.9642963310082754, test_error=-0.7662793099880219
	epoch #320: training_error=-0.9663448731104533, test_error=-0.768384113907814
	epoch #321: training_error=-0.959438270992703, test_error=-0.7648372054100037
	epoch #322: training_error=-0.96563617719

original cnn resize to 64 x 64

train 100 epoch

In [ ]:
training_src_folder = "../training_data/features/original/"
training_dst_folder = "../training_data/model/original/"


create_folder(training_dst_folder)
time = datetime.now(timezone('Asia/Bangkok')).strftime('log_date_%Y-%m-%d_time_%H-%M-%S')

path_log = training_dst_folder + time + ".txt"
file_writer = open(path_log, 'w')

training_features, training_labels = get_image(training_src_folder, training_dst_folder)
count = 1
for train_index, test_index in cv :
    round_log = "round => " + str(count) + "\n"
    train_log = "data trainng => " + str(train_index) + "\n"
    test_log = "data testing => " + str(test_index) + "\n"
    
    string_append_and_print(round_log + train_log + test_log, path_log)
    count +=1
    
    num_epochs = 100
    trainer, training_errors, test_errors = train(training_features, training_labels, training_dst_folder, path_log, train_index, test_index, num_epochs)
    
    path_model = training_dst_folder 
    name_model = "model{:.2f}".format(sum(test_errors)/len(test_errors)*100*-1)

    string_append_and_print("save model to => " + path_model + name_model + ".model", path_log)
    
    create_folder(path_model + "results/" + name_model)
    file = open( path_model + "results/" + name_model + "/training_result.txt", 'w')
    file.write(" ".join(map(str, training_errors)))
    file.close()

    file = open( path_model + "results/" + name_model + "/test_result.txt", 'w')
    file.write(" ".join(map(str, test_errors)))
    file.close()
    
    
    trainer.model.save(path_model + name_model + ".model")

train 200 epoch

In [ ]:
training_src_folder = "../training_data/features/original/"
training_dst_folder = "../training_data/model/original/"


create_folder(training_dst_folder)
time = datetime.now(timezone('Asia/Bangkok')).strftime('log_date_%Y-%m-%d_time_%H-%M-%S')

path_log = training_dst_folder + time + ".txt"
file_writer = open(path_log, 'w')

training_features, training_labels = get_image(training_src_folder, training_dst_folder)
count = 1
for train_index, test_index in cv :
    round_log = "round => " + str(count) + "\n"
    train_log = "data trainng => " + str(train_index) + "\n"
    test_log = "data testing => " + str(test_index) + "\n"
    
    string_append_and_print(round_log + train_log + test_log, path_log)
    count +=1
    
    num_epochs = 200
    trainer, training_errors, test_errors = train(training_features, training_labels, training_dst_folder, path_log, train_index, test_index, num_epochs)
    
    path_model = training_dst_folder 
    name_model = "model{:.2f}".format(sum(test_errors)/len(test_errors)*100*-1)

    string_append_and_print("save model to => " + path_model + name_model + ".model", path_log)
    
    create_folder(path_model + "results/" + name_model)
    file = open( path_model + "results/" + name_model + "/training_result.txt", 'w')
    file.write(" ".join(map(str, training_errors)))
    file.close()

    file = open( path_model + "results/" + name_model + "/test_result.txt", 'w')
    file.write(" ".join(map(str, test_errors)))
    file.close()
    
    
    trainer.model.save(path_model + name_model + ".model")

train 300 epoch

In [ ]:
training_src_folder = "../training_data/features/original/"
training_dst_folder = "../training_data/model/original/"


create_folder(training_dst_folder)
time = datetime.now(timezone('Asia/Bangkok')).strftime('log_date_%Y-%m-%d_time_%H-%M-%S')

path_log = training_dst_folder + time + ".txt"
file_writer = open(path_log, 'w')

training_features, training_labels = get_image(training_src_folder, training_dst_folder)
count = 1
for train_index, test_index in cv :
    round_log = "round => " + str(count) + "\n"
    train_log = "data trainng => " + str(train_index) + "\n"
    test_log = "data testing => " + str(test_index) + "\n"
    
    string_append_and_print(round_log + train_log + test_log, path_log)
    count +=1
    
    num_epochs = 300
    trainer, training_errors, test_errors = train(training_features, training_labels, training_dst_folder, path_log, train_index, test_index, num_epochs)
    
    path_model = training_dst_folder 
    name_model = "model{:.2f}".format(sum(test_errors)/len(test_errors)*100*-1)

    string_append_and_print("save model to => " + path_model + name_model + ".model", path_log)
    
    create_folder(path_model + "results/" + name_model)
    file = open( path_model + "results/" + name_model + "/training_result.txt", 'w')
    file.write(" ".join(map(str, training_errors)))
    file.close()

    file = open( path_model + "results/" + name_model + "/test_result.txt", 'w')
    file.write(" ".join(map(str, test_errors)))
    file.close()
    
    
    trainer.model.save(path_model + name_model + ".model")

train 500 epoch

In [ ]:
training_src_folder = "../training_data/features/original/"
training_dst_folder = "../training_data/model/original/"


create_folder(training_dst_folder)
time = datetime.now(timezone('Asia/Bangkok')).strftime('log_date_%Y-%m-%d_time_%H-%M-%S')

path_log = training_dst_folder + time + ".txt"
file_writer = open(path_log, 'w')

training_features, training_labels = get_image(training_src_folder, training_dst_folder)
count = 1
for train_index, test_index in cv :
    round_log = "round => " + str(count) + "\n"
    train_log = "data trainng => " + str(train_index) + "\n"
    test_log = "data testing => " + str(test_index) + "\n"
    
    string_append_and_print(round_log + train_log + test_log, path_log)
    count +=1
    
    num_epochs = 500
    trainer, training_errors, test_errors = train(training_features, training_labels, training_dst_folder, path_log, train_index, test_index, num_epochs)
    
    path_model = training_dst_folder 
    name_model = "model{:.2f}".format(sum(test_errors)/len(test_errors)*100*-1)

    string_append_and_print("save model to => " + path_model + name_model + ".model", path_log)
    
    create_folder(path_model + "results/" + name_model)
    file = open( path_model + "results/" + name_model + "/training_result.txt", 'w')
    file.write(" ".join(map(str, training_errors)))
    file.close()

    file = open( path_model + "results/" + name_model + "/test_result.txt", 'w')
    file.write(" ".join(map(str, test_errors)))
    file.close()
    
    
    trainer.model.save(path_model + name_model + ".model")

original augmentation cnn resize to 32 x 32

train 100 epoch

In [ ]:
training_src_folder = "../training_data/features/original_augment/"
training_dst_folder = "../training_data/model/original_augment/"


create_folder(training_dst_folder)
time = datetime.now(timezone('Asia/Bangkok')).strftime('log_date_%Y-%m-%d_time_%H-%M-%S')

path_log = training_dst_folder + time + ".txt"
file_writer = open(path_log, 'w')

training_features, training_labels = get_image(training_src_folder, training_dst_folder)count = 1
for train_index, test_index in cv :
    round_log = "round => " + str(count) + "\n"
    train_log = "data trainng => " + str(train_index) + "\n"
    test_log = "data testing => " + str(test_index) + "\n"
    
    string_append_and_print(round_log + train_log + test_log, path_log)
    count +=1
    
    num_epochs = 100
    trainer, training_errors, test_errors = train(training_features, training_labels, training_dst_folder, path_log, train_index, test_index, num_epochs)
    
    path_model = training_dst_folder 
    name_model = "model{:.2f}".format(sum(test_errors)/len(test_errors)*100*-1)

    string_append_and_print("save model to => " + path_model + name_model + ".model", path_log)
    
    create_folder(path_model + "results/" + name_model)
    file = open( path_model + "results/" + name_model + "/training_result.txt", 'w')
    file.write(" ".join(map(str, training_errors)))
    file.close()

    file = open( path_model + "results/" + name_model + "/test_result.txt", 'w')
    file.write(" ".join(map(str, test_errors)))
    file.close()
    
    
    trainer.model.save(path_model + name_model + ".model")

train 200 epoch

In [ ]:
training_src_folder = "../training_data/features/original_augment/"
training_dst_folder = "../training_data/model/original_augment/"


create_folder(training_dst_folder)
time = datetime.now(timezone('Asia/Bangkok')).strftime('log_date_%Y-%m-%d_time_%H-%M-%S')

path_log = training_dst_folder + time + ".txt"
file_writer = open(path_log, 'w')

training_features, training_labels = get_image(training_src_folder, training_dst_folder)count = 1
for train_index, test_index in cv :
    round_log = "round => " + str(count) + "\n"
    train_log = "data trainng => " + str(train_index) + "\n"
    test_log = "data testing => " + str(test_index) + "\n"
    
    string_append_and_print(round_log + train_log + test_log, path_log)
    count +=1
    
    num_epochs = 200
    trainer, training_errors, test_errors = train(training_features, training_labels, training_dst_folder, path_log, train_index, test_index, num_epochs)
    
    path_model = training_dst_folder 
    name_model = "model{:.2f}".format(sum(test_errors)/len(test_errors)*100*-1)

    string_append_and_print("save model to => " + path_model + name_model + ".model", path_log)
    
    create_folder(path_model + "results/" + name_model)
    file = open( path_model + "results/" + name_model + "/training_result.txt", 'w')
    file.write(" ".join(map(str, training_errors)))
    file.close()

    file = open( path_model + "results/" + name_model + "/test_result.txt", 'w')
    file.write(" ".join(map(str, test_errors)))
    file.close()
    
    
    trainer.model.save(path_model + name_model + ".model")

train 300 epoch

In [ ]:
training_src_folder = "../training_data/features/original_augment/"
training_dst_folder = "../training_data/model/original_augment/"


create_folder(training_dst_folder)
time = datetime.now(timezone('Asia/Bangkok')).strftime('log_date_%Y-%m-%d_time_%H-%M-%S')

path_log = training_dst_folder + time + ".txt"
file_writer = open(path_log, 'w')

training_features, training_labels = get_image(training_src_folder, training_dst_folder)count = 1
for train_index, test_index in cv :
    round_log = "round => " + str(count) + "\n"
    train_log = "data trainng => " + str(train_index) + "\n"
    test_log = "data testing => " + str(test_index) + "\n"
    
    string_append_and_print(round_log + train_log + test_log, path_log)
    count +=1
    
    num_epochs = 300
    trainer, training_errors, test_errors = train(training_features, training_labels, training_dst_folder, path_log, train_index, test_index, num_epochs)
    
    path_model = training_dst_folder 
    name_model = "model{:.2f}".format(sum(test_errors)/len(test_errors)*100*-1)

    string_append_and_print("save model to => " + path_model + name_model + ".model", path_log)
    
    create_folder(path_model + "results/" + name_model)
    file = open( path_model + "results/" + name_model + "/training_result.txt", 'w')
    file.write(" ".join(map(str, training_errors)))
    file.close()

    file = open( path_model + "results/" + name_model + "/test_result.txt", 'w')
    file.write(" ".join(map(str, test_errors)))
    file.close()
    
    
    trainer.model.save(path_model + name_model + ".model")